# Make datasets

In [9]:
import glob
import pandas as pd
import json

# Find all JSON files in the specified directory
json_files_train = glob.glob('/Users/theo/Desktop/MVA/Second_Semestre/LLM/Projet-final/ECHR_Dataset/EN_train/*.json')
json_files_dev = glob.glob('/Users/theo/Desktop/MVA/Second_Semestre/LLM/Projet-final/ECHR_Dataset/EN_dev/*.json')
json_files_test = glob.glob('/Users/theo/Desktop/MVA/Second_Semestre/LLM/Projet-final/ECHR_Dataset/EN_test/*.json')
rows_train = []
rows_dev = []
rows_test = []

for file in json_files_train:
    with open(file, 'r') as f:
        data_train  = json.load(f)
        # If needed, process certain fields (e.g., join lists) here:
        # data['TEXT'] = ' '.join(data['TEXT'])
        rows_train.append(data_train)

# Create the global DataFrame
df_train = pd.DataFrame(rows_train)


for file in json_files_dev:
    with open(file, 'r') as f:
        data_dev = json.load(f)
        # If needed, process certain fields (e.g., join lists) here:
        # data['TEXT'] = ' '.join(data['TEXT'])
        rows_dev.append(data_dev)

# Create the global DataFrame
df_dev = pd.DataFrame(rows_dev)

for file in json_files_test:
    with open(file, 'r') as f:
        data_test = json.load(f)
        # If needed, process certain fields (e.g., join lists) here:
        # data['TEXT'] = ' '.join(data['TEXT'])
        rows_test.append(data_test)
 
# Create the global DataFrame
df_test = pd.DataFrame(rows_test)




In [15]:
# Get binary column for each dataset for violation
# List of columns to check
columns_to_check = [
    'VIOLATED_ARTICLES', 'VIOLATED_PARAGRAPHS', 'VIOLATED_BULLETPOINTS'
]

# Create the new binary column
df_train['VIOLATED'] = df_train[columns_to_check].apply(
    lambda row: 1 if any(len(item) > 0 for item in row) else 0,
    axis=1

)

df_dev['VIOLATED'] = df_dev[columns_to_check].apply(
    lambda row: 1 if any(len(item) > 0 for item in row) else 0,
    axis=1

)

df_test['VIOLATED'] = df_test[columns_to_check].apply(
    lambda row: 1 if any(len(item) > 0 for item in row) else 0,
    axis=1

)


In [16]:
df_train

,ITEMID,LANGUAGEISOCODE,RESPONDENT,BRANCH,DATE,DOCNAME,IMPORTANCE,CONCLUSION,JUDGES,TEXT,VIOLATED_ARTICLES,VIOLATED_PARAGRAPHS,VIOLATED_BULLETPOINTS,NON_VIOLATED_ARTICLES,NON_VIOLATED_PARAGRAPHS,NON_VIOLATED_BULLETPOINTS,binary violates,VIOLATED
0,001-60310,ENG,TUR,CHAMBER,2002,CASE OF SABUKTEKIN v. TURKEY,1,No violation of Art. 2 concerning the death of...,Matti Pellonpää,[7. On 28 September 1994 the applicant's husba...,[],[],[],"[13, 2]",[],[],0,0
1,001-93292,ENG,POL,CHAMBER,2009,CASE OF GRZEGORZ HULEWICZ v. POLAND (No. 2),4,No violation of Article 5 - Right to liberty a...,David Thór Björgvinsson;Giovanni Bonello;Lech ...,[8. The applicant was born in 1974 and lives i...,[],[],[],[5],[],[],0,0
2,001-93768,ENG,SVK,CHAMBER,2009,CASE OF DVORACEK AND DVORACKOVA v. SLOVAKIA,3,Remainder inadmissible;Violation of Art. 2 (pr...,David Thór Björgvinsson;Giovanni Bonello;Ján Š...,"[5. The first applicant, Mr Ivan Dvořáček, was...","[2, 6]",[6-1],[],[],[],[],1,1
3,001-82483,ENG,TUR,CHAMBER,2007,CASE OF MAHMUT ASLAN v. TURKEY,3,"Preliminary objections dismissed (victim, non-...",Nicolas Bratza,[4. The applicant was born in 1959 and lives i...,"[13, 6]",[6-1],[],[],[],[],1,1
4,001-61852,ENG,LIE,CHAMBER,2004,CASE OF FROMMELT v. LIECHTENSTEIN,3,Violation of Art. 5-4;Non-pecuniary damage - f...,Georg Ress;Mark Villiger,"[6. The applicant was born in 1946., 7. On 14 ...",[5],[5-4],[],[],[],[],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,001-75243,ENG,SVN,CHAMBER,2006,CASE OF GASHI v. SLOVENIA,4,Violation of Art. 6-1;Violation of Art. 13;Non...,David Thór Björgvinsson;John Hedigan,[5. The applicant was born in 1943 and lives i...,"[13, 6]",[6-1],[],[],[],[],1,1
7096,001-23423,ENG,SVK,ADMISSIBILITY,2003,VACLAVIK v. SLOVAKIA,4,Inadmissible,Nicolas Bratza,"[The applicant, Mr Dušan Václavík, is a Slovak...",[],[],[],[],[],[],0,0
7097,001-76896,ENG,SRB,CHAMBER,2006,CASE OF MATIJASEVIC v. SERBIA,1,Violation of Art. 6-2;Remainder inadmissible;N...,András Baka;Antonella Mularoni;Elisabet Fura;I...,[4. The applicant was born in 1976 and is curr...,[6],[6-2],[],[],[],[],1,1
7098,001-78471,ENG,SVK,ADMISSIBILITY,2006,BALAZ AND OTHERS v. SLOVAKIA,4,Inadmissible,Nicolas Bratza,[The applicants are relatives. They are all Sl...,[],[],[],[],[],[],0,0


# Load models

Bert Tiny

In [13]:
# Load model directly
from transformers import AutoModel
model_bert_tiny = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

In [17]:
import torch
import torch.nn as nn
from transformers import AutoModel

class BertClassifier(nn.Module):
    def __init__(self, dropout_rate=0.1):
        super(BertClassifier, self).__init__()
        # Load the pre-trained model
        self.bert = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
        self.dropout = nn.Dropout(dropout_rate)
        # Classification head: Dense layer with tanh activation followed by another Dense layer
        self.dense_tanh = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
        self.tanh = nn.Tanh()
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)  # binary classification -> 2 classes
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_ids, attention_mask):
        # Get outputs from BERT; use pooler_output (or outputs[1] if pooler_output is absent)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output if hasattr(outputs, 'pooler_output') else outputs[1]
        # Apply dropout, tanh activation and another dropout
        x = self.dropout(pooled_output)
        x = self.dense_tanh(x)
        x = self.tanh(x)
        x = self.dropout(x)
        # Compute logits and probabilities
        logits = self.classifier(x)
        probs = self.softmax(logits)
        return probs


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
import pandas as pd

# Assume df_train_dev is your combined training+development DataFrame.
df_train_dev = df_train.append(df_dev, ignore_index=True)
# It must contain the columns:
#   "CONCLUSION" (input text)
#   "VIOLATED" (target binary label; e.g., 0 or 1)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

class TextDataset(Dataset):
    def __init__(self, dataframe, text_column, label_column, max_length=128):
        self.dataframe = dataframe
        self.text_column = text_column
        self.label_column = label_column
        self.max_length = max_length
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx][self.text_column]
        label = self.dataframe.iloc[idx][self.label_column]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),       # shape: (max_length,)
            'attention_mask': encoding['attention_mask'].squeeze(),  # shape: (max_length,)
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create dataset and dataloader
dataset = TextDataset(df_train_dev, text_column='CONCLUSION', label_column='VIOLATED', max_length=128)
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the model and move to device
model = BertClassifier(dropout_rate=0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Set up training parameters
# Grid-search example: iterate over learning rates in {2e-5, 3e-5, 4e-5, 5e-5}
# For demonstration, we use learning_rate = 2e-5. To grid search, wrap the following in a loop.
learning_rate = 2e-5
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 30
total_steps = len(dataloader) * num_epochs

# Scheduler with 100k warm-up steps (ensure warm-up steps do not exceed total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100000, num_training_steps=total_steps)

loss_fn = nn.CrossEntropyLoss()

# Early stopping parameters
patience = 2
best_loss = float('inf')
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")
    
    # Early stopping check (using training loss as a proxy; ideally, use a dev set loss)
    if avg_loss < best_loss:
        best_loss = avg_loss
        epochs_no_improve = 0
        # Save the best model checkpoint
        torch.save(model.state_dict(), "best_model.pt")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered")
            break


/var/folders/p1/bzcnw1ns6b544bv2hbtjsmmr0000gn/T/ipykernel_27105/707452331.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train_dev = df_train.append(df_dev, ignore_index=True)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Epoch 1/30 - Loss: 0.6877
Epoch 2/30 - Loss: 0.6831
Epoch 3/30 - Loss: 0.6711
Epoch 4/30 - Loss: 0.6508
Epoch 5/30 - Loss: 0.6200
Epoch 6/30 - Loss: 0.5834
Epoch 7/30 - Loss: 0.5515
Epoch 8/30 - Loss: 0.5238
Epoch 9/30 - Loss: 0.5002
Epoch 10/30 - Loss: 0.4776
Epoch 11/30 - Loss: 0.4550
Epoch 12/30 - Loss: 0.4291
Epoch 13/30 - Loss: 0.4072
Epoch 14/30 - Loss: 0.3934
Epoch 15/30 - Loss: 0.3816
Epoch 16/30 - Loss: 0.3741
Epoch 17/30 - Loss: 0.3647
Epoch 18/30 - Loss: 0.3574
Epoch 19/30 - Loss: 0.3525
Epoch 20/30 - Loss: 0.3510
Epoch 21/30 - Loss: 0.3469
Epoch 22/30 - Loss: 0.3457
Epoch 23/30 - Loss: 0.3428
Epoch 24/30 - Loss: 0.3415
Epoch 25/30 - Loss: 0.3397
Epoch 26/30 - Loss: 0.3399
Epoch 27/30 - Loss: 0.3382
Epoch 28/30 - Loss: 0.3360
Epoch 29/30 - Loss: 0.3358
Epoch 30/30 - Loss: 0.3339


In [21]:
import torch
from torch.utils.data import DataLoader

# Assume df_test is your test DataFrame with the same "CONCLUSION" and "VIOLATED" columns.
test_dataset = TextDataset(df_test, text_column='CONCLUSION', label_column='VIOLATED', max_length=128)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Load the best model checkpoint
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # Get predicted labels from softmax probabilities
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)
        
accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy:.4f}")


/var/folders/p1/bzcnw1ns6b544bv2hbtjsmmr0000gn/T/ipykernel_27105/158280306.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.p

Test Accuracy: 0.9713


Legal Bert Base uncased

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")